In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
from src.data_sources import process_all, arr2csv
from src.midi_transform import *

In [3]:
import traceback
import time

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v13'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
source_dir = 'midi_sources'
out_dir = 'midi_encode'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v13/metadata/midi_sources.csv'),
 PosixPath('data/midi/v13/piano_duet/piano_duet.csv'))

In [7]:
num_comps = 2 # note, duration
cutoff = 6 # max instruments
max_dur = 128

### Encoding midi to numpy

In [6]:
df = pd.read_csv(source_csv); df.head()

NameError: name 'pd' is not defined

In [5]:
all_records = df.to_dict(orient='records'); len(all_records)

NameError: name 'df' is not defined

In [10]:
def transform_func(metadata):
    result = metadata.copy()
    start = time.time()
    
    # Part 1. Compress tracks/instruments
    if not isinstance(metadata.get('midi'), str): return None
    
    input_path = version_path/metadata['midi']
    extension = input_path.suffix.lower()
    if not input_path.exists(): 
        print('Input path does not exist:', input_path, metadata)
        return None
    
    # Get outfile and check if it exists
    out_file = Path(str(input_path).replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file = out_file.with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): 
        result['numpy'] = str(out_file.relative_to(version_path))
        return result
    
    if duet_only:
        try: 
            if num_piano_tracks(input_path) not in [1, 2] : return None
        except Exception: return None
    
    try: input_file = compress_midi_file(input_path, cutoff=cutoff) # remove non note tracks and standardize instruments
    except music21.midi.MidiException as e:
        print('Error parsing midi', input_path, e)
        return None
    if not input_file: return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream, max_dur=max_dur, flat=False) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
#     chordarr = stream2chordarr(stream, max_dur=max_dur, flat=(extension != '.mid')) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
#         print(traceback.format_exc())
        return None
    
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
    if delta_trim > 300: 
        print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
        return None
    chordarr = chord_short
    
    # Part 3. Chord array to numpy
    seq = chordarr2seq(chordarr)
    if len(seq) < 32:
        print('Sequence too short:', len(seq), input_path)
        return None
    
#     category = ENC_OFFSET + max_dur + sources.index(metadata['source']) + 1
    category = source2encidx(metadata['source'], max_dur=max_dur)
    npenc = seq2npenc(seq, num_comps=num_comps, category=category)
    if (npenc[1:,1] > max_dur+ENC_OFFSET).any(): 
        print('npenc exceeds max 128 duration:', input_path)
        return None
    
    if ((npenc[...,0] > ENC_OFFSET) & ((npenc[...,0] < 12+ENC_OFFSET) | (npenc[...,0] >= 127-12+ENC_OFFSET))).any(): 
        print('npenc out of note range 12 - 116:', input_path)
        return None
    
    
    np.save(out_file, npenc)
    
    elapsed_time = time.time() - start
    result['sec_elapsed_tfm'] = elapsed_time
    if elapsed_time > 60:
        print('Super long file:', elapsed_time, input_path)
    
    result['numpy'] = str(out_file.relative_to(version_path))
    return result

In [11]:
def try_transform_func(metadata):
    try:
        return transform_func(metadata)
    except Exception:
        print(traceback.format_exc())
        return None

In [12]:
# # sanity check
import random
for r in random.sample(all_records, 10):
    transform_func(r)
        

In [13]:
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

In [ ]:
processed = process_all(try_transform_func, all_records, timeout=300, timeout_func=timeout_func)

Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/y/yes/heart-of-the-sunrise/intro_key_original.mid
Sequence too short: 29 data/midi/v13/midi_sources/hooktheory/pianoroll/j/jeff-liu/steven-and-the-crystal-gems/outro_key_original.mid
Sequence too short: 29 data/midi/v13/midi_sources/hooktheory/pianoroll/j/jessica-simpson/part-of-your-world/bridge_key_original.mid
Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/j/jamiroquai/canned-heat/intro_key_original.mid
Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/l/lildeucedeuce/mine-turtle/intro_key_original.mid
Sequence too short: 17 data/midi/v13/midi_sources/hooktheory/pianoroll/f/frankie-valli/the-night/intro_key_original.mid
Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/n/nobuo-uematsu/final-fantasy-vi---dancing-mad-part-3/bridge_key_original.mid
Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/n/nobuo-uematsu/final-fantasy-vii

npenc exceeds max 128 duration: data/midi/v13/midi_sources/midiworld/named_midi/Benjy_shelton__-_Conker's_bad_fur_day_big_poop.mid
npenc exceeds max 128 duration: data/midi/v13/midi_sources/midiworld/named_midi/Led_Zeppelin_-_In_the_Evening.mid
npenc exceeds max 128 duration: data/midi/v13/midi_sources/midiworld/named_midi/Korn_-_Kill_You.mid
Could not encode to chordarr: data/midi/v13/midi_sources/midiworld/named_midi/Chubby_Checker_-_Lets_Twist_Again.mid index 1727 is out of bounds for axis 0 with size 1727
Removed 544 rests from data/midi/v13/midi_sources/midiworld/named_midi/Metallica_-_The_Call_Of_Ktulu.mid.
npenc exceeds max 128 duration: data/midi/v13/midi_sources/midiworld/named_midi/Silverchair_-_Lie_to_Me.mid
Sequence too short: 0 data/midi/v13/midi_sources/midiworld/named_midi/Cypress_Hill_-_Rock_Superstar.mid
Sequence too short: 17 data/midi/v13/midi_sources/midiworld/named_midi/Alexandros_Mousafeiris_-_Sanchit's_Tune.mid
Removed 384 rests from data/midi/v13/midi_sources/mi

In [ ]:
arr2csv(processed, out_csv); len(processed)

In [29]:
df = pd.read_csv(out_csv); df.head()

189846